In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm

In [2]:
data_df = pd.read_csv('SIR_simulation.csv')
data_df.head()

,time,I,pol,r,ar,cc,kind,net_type
0,0,5.00,0.1,0.1,724,0.049,mean,static
1,0,0.00,0.1,0.1,33,0.003,std,static
2,1,11.28,0.1,0.1,724,0.049,mean,static
3,1,2.74,0.1,0.1,33,0.003,std,static
4,2,44.44,0.1,0.1,724,0.049,mean,static


In [3]:
mask1 = (data_df.net_type == 'static') & (data_df.kind == 'mean')
mask2 = (data_df.net_type == 'static') & (data_df.kind == 'std')
mask3 = (data_df.net_type == 'dynamic') & (data_df.kind == 'mean')
mask4 = (data_df.net_type == 'dynamic') & (data_df.kind == 'std')

static_mean = data_df[mask1].filter(items=['time', 'I'])
static_std = data_df[mask2].filter(items=['time', 'I'])
dynamic_mean = data_df[mask3].filter(items=['time', 'I'])
dynamic_std = data_df[mask4].filter(items=['time', 'I'])

In [ ]:
colors = cm.get_cmap('winter', 2)
fig, ax = plt.subplots(nrows=9, ncols=9, figsize=(20,20))
pol_list = np.arange(0.1, 1., 0.1)
r_list = np.arange(0.1, 1., 0.1)
for i, pol in enumerate(pol_list):
    for j, r in enumerate(r_list):
        mask1 = (static_mean.r == r) & (static_mean.pol == pol)
        mask2 = (static_std.r == r) & (static_std.pol == pol)
        mask3 = (dynamic_mean.r == r) & (dynamic_mean.pol == pol)
        mask4 = (dynamic_std.r == r) & (dynamic_std.pol == pol)
        ax_stat_mean = static_mean[mask1].set_index('time')
        ax_stat_std = static_std[mask2].set_index('time')
        ax_dyn_mean = dynamic_mean[mask3].set_index('time')
        ax_dyn_std = dynamic_std[mask4].set_index('time')

        ax[i,j].set_title(f'polarization = {pol}, vaccination rate = {r}')
        ax.set_xlabel('time')
        ax.set_ylabel('Prevalence')
        ax[i,j].fill_between(ax_stat_mean.index, ax_stat_mean - ax_stat_std, ax_stat_mean + ax_stat_std, col)

